In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Machine Learning models application to imdb review dataset

In [1]:
# # # # # #

In [2]:
import numpy as np
import pandas as pd

In [3]:
# load 'movie_data.csv' to pandas dataframe.
# Show first 5 reviews
df = pd.read_csv("/content/drive/My Drive/datasets/movie_data.csv")
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [4]:
# use nltk library to identify stopwords in english
# apply regular expressions to remove everything 
#(stop words, pantuations, symbols and so on) except necessary words. 
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')
df['review'] = df['review'].str.lower()

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop])

df['review'] = df['review'].apply(stopwords)
df['review'] = df['review'].str.replace(r'[^\w\s]+', '')

display(df[:12])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment
0,went saw movie last night coaxed friends mine ...,1
1,actor turned director bill paxton follows prom...,1
2,recreational golfer knowledge sports history p...,1
3,saw film sneak preview delightful cinematograp...,1
4,bill paxton taken true story 1913 us golf open...,1
5,saw film september 1st 2005 indianapolis one j...,1
6,maybe im reading much wonder much hand hongshe...,1
7,felt film many good qualities cinematography c...,1
8,movie amazing fact real people portray real li...,1
9,quitting may much exiting preordained identity...,1


In [5]:
# Show the reviews to be sure that you are going in the right direction
des = df['review'].describe()
des


count                                                 50000
unique                                                49579
top       loved todays show variety solely cooking which...
freq                                                      5
Name: review, dtype: object

In [6]:
# Find the word counts in the reviews
# remove the words that appears less than 10 times in the reviews
# they do not help us for the classification.
from collections import Counter
from itertools import chain

v = df['review'].str.split().tolist()
c = Counter(chain.from_iterable(v))
df['review'] = [' '.join([j for j in i if c[j] > 9]) for i in v]
display(df[:12])
#display(c)

,review,sentiment
0,went saw movie last night friends mine ill adm...,1
1,actor turned director bill paxton follows prom...,1
2,recreational knowledge sports history pleased ...,1
3,saw film sneak preview delightful cinematograp...,1
4,bill paxton taken true story 1913 us golf open...,1
5,saw film september 1st 2005 one judges heartla...,1
6,maybe im reading much wonder much hand develop...,1
7,felt film many good qualities cinematography c...,1
8,movie amazing fact real people portray real li...,1
9,quitting may much exiting identity drug withdr...,1


In [7]:
# split the dataset into  train and test datasets 
# train dataset will be %80 and test set will be %20
from sklearn import model_selection, preprocessing, feature_extraction, linear_model, naive_bayes
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=1)



In [8]:
# create input(X) and output (y) vectors for train and test datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
encoder = preprocessing.LabelEncoder()

vectorizer.fit(train_x)

train_xCount = vectorizer.transform(train_x)
test_xCount = vectorizer.transform(test_x)
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)



In [13]:
print(train_y.shape)

(40000,)


In [14]:
test_y[0:5]

array([1, 1, 1, 0, 1])

In [9]:
wordVector = TfidfVectorizer()
wordVector.fit(train_x)
train_xWord = wordVector.transform(train_x).astype('float32')
test_xWord = wordVector.transform(test_x).astype('float32')
wordVector.get_feature_names()[0:5]


['007', '00s', '01', '010', '05']

In [10]:
train_xWord = train_xWord.toarray()
#np.save('train_xWord.npy', train_xWord)

In [11]:
test_xWord = test_xWord.toarray()

In [18]:
# show the dimensions and sample data from train and test datasets 
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

print(train_x[:5])
print(train_y[:5])
print(test_x[:5])
print(test_y[:5])

(40000,)
(40000,)
(10000,)
(10000,)
18165    dr lucio fulci lucio fulci director gory horro...
36059    lot crap coming hollywood br a friend sends mo...
13242    one awaited moviei thought himesh bit acting a...
32985    another exquisite taste superhero movie end he...
41133    iberia nice see tv see silver screen lot dance...
Name: review, dtype: object
[0 1 0 1 0]
26247    started watching series cable idea addictive w...
35067    steve biko black activist tried resist white m...
34590    short comment flick go pick up chances going p...
16668    serious horror fan get certain marketing ploys...
12196    robert cummings laraine day jean muir star and...
Name: review, dtype: object
[1 1 1 0 1]


In [42]:
# create Neural Network model that may contain one or more hidden layers
# train the model and print out accuracy and loss
# predict first 5 reviews of test datasets
# compare these predictions with the actual labels

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

train_y2 = tf.keras.utils.to_categorical(train_y, num_classes=2)
test_y2 = tf.keras.utils.to_categorical(test_y, num_classes=2)
batch_size = 256
epochs = 10
num_classes = 2

model = Sequential()
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(train_xWord, train_y2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_xWord, test_y2))
score = model.evaluate(test_xWord, test_y2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
157/157 [==============================] - 17s 107ms/step - loss: 0.3842 - accuracy: 0.8630 - val_loss: 0.2539 - val_accuracy: 0.8979
Epoch 2/10
157/157 [==============================] - 17s 106ms/step - loss: 0.1636 - accuracy: 0.9433 - val_loss: 0.2442 - val_accuracy: 0.8999
Epoch 3/10
157/157 [==============================] - 17s 108ms/step - loss: 0.1001 - accuracy: 0.9706 - val_loss: 0.2695 - val_accuracy: 0.8942
Epoch 4/10
157/157 [==============================] - 17s 108ms/step - loss: 0.0622 - accuracy: 0.9854 - val_loss: 0.3020 - val_accuracy: 0.8902
Epoch 5/10
157/157 [==============================] - 17s 106ms/step - loss: 0.0374 - accuracy: 0.9937 - val_loss: 0.3413 - val_accuracy: 0.8873
Epoch 6/10
157/157 [==============================] - 17s 109ms/step - loss: 0.0221 - accuracy: 0.9977 - val_loss: 0.3737 - val_accuracy: 0.8848
Epoch 7/10
157/157 [==============================] - 17s 106ms/step - loss: 0.0134 - accuracy: 0.9992 - val_loss: 0.4038 - val_ac

In [34]:
actualFive = df['sentiment'][0:5]
firstFive1 = df['review'][0:5]
wordVector.fit(train_x)
firstFive1 = wordVector.transform(firstFive1).astype('float32')
firstFive1 = firstFive1.toarray()

print("Actual values    : ",np.argmax(test_y2[0:5,:], axis=1))
print("Predicted values : ",np.argmax(model.predict(test_xWord[0:5,:]), axis=1))

Actual values    :  [1 1 1 0 1]
Predicted values :  [1 1 1 0 1]


In [12]:
# classification using sklearn.linear_model.LogisticRegression module
# train the model and print out accuracy and loss
# predict first 5 reviews of test datasets
# compare these preditions with the actual labels

logReg = linear_model.LogisticRegression(max_iter = 40000)
logRegModel = logReg.fit(train_xCount, train_y)

accuracyLR = logReg.score(test_xCount, test_y)
print("Accuracy: ",accuracyLR)

Accuracy:  0.8835


In [30]:
print("Actual values    : ",test_y[0:5])
print("Predicted values : ",logRegModel.predict(test_xCount[0:5]))

Actual values    :  [1 1 1 0 1]
Predicted values :  [1 1 1 0 0]


In [32]:
# classification using sklearn.naive_bayes.GaussianNB
# train the model and print out accuracy and loss
# predict first 5 reviews of test datasets
# compare these preditions with the actual labels
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_xWord, train_y)

#Predict the response for test dataset
y_pred = gnb.predict(test_xWord)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

accuracyNB = metrics.accuracy_score(test_y, y_pred)
print("Accuracy:",accuracyNB)



Accuracy: 0.7225


In [39]:
print("Actual values    : ",test_y[0:5])
print("Predicted values : ",gnb.predict(test_xWord[0:5,:]))

Actual values    :  [1 1 1 0 1]
Predicted values :  [1 1 1 1 1]


In [ ]:
# use sklearn.decomposition.PCA to reduce the dimensions of input(X) 
from sklearn.decomposition import PCA
#xx = np.load('train_xWord.npy')
pca = PCA(n_components = 100)
fit = pca.fit_transform(train_xWord)
fitTest = pca.fit_transform(test_xWord)

print('Original number of features:', train_xWord.shape[1])
print('Reduced number of features:', fit.shape[1])


Original number of features: 30714
Reduced number of features: 100


In [ ]:
np.save('reducedX.npy', fit)
np.save('reducedXtest.npy', fit)

In [ ]:
# After having less dimensional input, KNN application
# check whether KNN provides reasonable performance
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
#fit = np.load('/content/drive/My Drive/datasets/reducedX.npy')
#fitTest = np.load('/content/drive/My Drive/datasets/reducedXtest.npy')
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(fit,train_y)
accuracyKNN = knn.score(fitTest, test_y)
print("Accuracy:",accuracyKNN)

Accuracy: 0.6741


In [46]:
# Comparison of models
print("The most successful model is Logistic Regression model. It's accuracy is                             : ", accuracyLR)
print("The second successful model is Neural Network model with a pretty small difference. It's accuracy is : ", score[1])
print("The third successful model is Gaussian Naive Bayes model. It's accuracy is                           : ", accuracyNB)
print("The model that has the worst accuracy is KNN. It's accuracy is                                       :  0.6741")

The most successful model is Logistic Regression model. It's accuracy is                             :  0.8835
The second successful model is Neural Network model with a pretty small difference. It's accuracy is :  0.8822000026702881
The third successful model is Gaussian Naive Bayes model. It's accuracy is                           :  0.7225
The model that has the worst accuracy is KNN. It's accuracy is                                       :  0.6741
